# Нейроэволюционный алгоритм

## Загружаем и подготавливаем данные для работы

### Строки, не несущие информацию были удалены

In [ ]:
import datatable as dt
data = dt.fread('cancer1.dt')
del data[0:7,:]

## Преробразуем данные в формат pandas.DataFrame 

In [ ]:
import pandas as pd
df = pd.DataFrame(data.to_pandas())
df

## Разделяем данные на входные и требуемые выходные значения

In [ ]:
outputs = df[['C9','C10']] \
                    .iloc[:,0]\
                    .astype('int64')

outputs

In [ ]:
inputs = df.iloc[:, 0:9].copy().astype('float64')
inputs

## Разбиваем выборку на тренировочную и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, train_size = 0.8, random_state = 3)

In [ ]:
inputs_train.info()
inputs_test.info()

In [ ]:
# Класс для ИНС (пока нерабочий)
class NeuralNetwork:
  fitness = 0.0
  def __init__(self, shape):
    self.neurons = []
    self.w = []
    self.b = []

  def sigmoid(x):
    s = np.array(x)
    s = 1/(1+np.exp(-s))
    return s

  def forward_propagation(self, inputs):
      if len(inputs) == len(self.neurons[0]):
      self.neurons[0][:]=data[:]

      for i in range(1,len(self.neurons)):
        for j in range(len(self.neurons[i])):
          self.neurons[i][j]=sigmoid(np.dot(self.neurons[i-1],self.w[i-1][j])+self.b[i-1][j])
      return self.neurons[-1]

  def mutation(self):
      pass



In [ ]:
# Гиперпараметры

EPOCHS = 500
SHAPE_NN = (8,7,6,1)
NNS_IN_POPULATION_COUNTS = 10

CORRELATION_CONSTANT = 0.4
BLX_A = 0.5
MUTATION_COUNTS = 2


In [ ]:
# инициализация начальной популяции

for count in range(COUNT_NNS_IN_POPULATION):
  neural_network = NeuralNetwork(SHAPE_NN)
  population.append(neural_network)

error_f = np.zeros(len(population))